### **Import OneLakeRemoteOperations Module from GitHub**
**Reference:** [Microsoft Fabric: OneLake Remote Operations](https://github.com/renan-peres/fabric-onelake-remote-operations)

In [1]:
import requests

# Fetch the main.py file from the GitHub repository
url = "https://raw.githubusercontent.com/renan-peres/fabric-onelake-remote-operations/main/main.py"
response = requests.get(url)
exec(response.text)

### **Assign OneLakeRemoteOperations and FabricAPIOperations to Variables, & Get Authentication Token**

In [ ]:
%run main.py

# Initiate OneLakeRemoteOperations and get the token
onelake_ops = OneLakeRemoteOperations()

# Get Authentication token from OneLakeRemoteOperations and Save into 'token_store.json'
token = onelake_ops.get_bearer_token()

# Initiate FabricAPIOperations and call the method
fabric_ops = FabricAPIOperations()

## **Import/Create Notebook**
**Reference:** [Manage and Execute Notebooks in Fabric with APIs](https://learn.microsoft.com/en-us/fabric/data-engineering/notebook-public-api)

In [ ]:
# Create Notebook in Workspace
fabric_ops.import_notebook_to_fabric(
    token=token,
    upload_from="github",  # ('local', 'lakehouse', or 'github')
    # source_path="Files/GitHub/notebooks/R in Spark/R_in_PySpark_Notebooks.ipynb", # lakehouse
    source_path="https://github.com/renan-peres/Polars-Cookbook/blob/main/Chapter07/ch07.ipynb", # github
    # source_path="test.ipynb", # local
    known_lakehouses=["3b9aef1c-59f2-47d9-a8b2-d8b234536427", "46389222-328e-4e65-aa06-02a380dd60d8"], # [LH_Test (Default), LH_bronze]
    # Optional parameters:
    # default_lakehouse_id="46389222-328e-4e65-aa06-02a380dd60d8", # LH_bronze
    # default_lakehouse_workspace_id="custom_workspace_id",
    # environment_id="custom_environment_id",
    # environment_workspace_id="custom_environmet_workspace_id"
)

## **Run Spark Notebook**
**Reference:** [Manage and Execute Notebooks in Fabric with APIs](https://learn.microsoft.com/en-us/fabric/data-engineering/notebook-public-api)

In [ ]:
%run get-bearer-token.py
import requests
import os
import json

def run_notebook_job(artifact_id: str, workspace_id: str, lakehouse_name: str, lakehouse_id: str, token: str) -> str:
    endpoint = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{artifact_id}/jobs/instances?jobType=RunNotebook"
    payload = {
        "executionData": {
                "defaultLakehouse": {
                    "name": lakehouse_name,
                    "id": lakehouse_id,
                },
                "useStarterPool": True
            }
        }
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    response = requests.post(endpoint, json=payload, headers=headers)
    if response.status_code == 202:
        return response.headers.get("Location")
    else:
        print(f"Failed to trigger table maintenance job. Status code: {response.status_code}, Response text: {response.text}")
        return None

# Microsoft Fabric Details from Environment Variables
workspace_id=os.getenv("WORKSPACE_ID")
artifact_id = os.getenv("ARTIFACT_ID")
lakehouse_id=os.getenv("LAKEHOUSE_ID")
lakehouse_name = os.getenv("LAKEHOUSE_NAME")

# Get the token using the class method
onelake_ops = OneLakeRemoteOperations()
token = onelake_ops.get_bearer_token()

# Trigger the pipeline job
run_notebook_job(workspace_id = workspace_id, 
                 artifact_id=artifact_id, 
                 lakehouse_id = lakehouse_id, 
                 lakehouse_name=lakehouse_name, 
                 token=token)

## **Run Data Pipeline**
**Reference:** [Microsoft Fabric data pipeline public REST API](https://learn.microsoft.com/en-us/fabric/data-factory/pipeline-rest-api)

In [ ]:
%run main.py
import requests
import os

def trigger_pipeline_job(workspace_id: str, pipeline_id: str, token: str) -> str:
    endpoint = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{pipeline_id}/jobs/instances?jobType=Pipeline"
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    response = requests.post(endpoint, headers=headers)
    if response.status_code == 202:
        return response.headers.get("Location")
    else:
        response.raise_for_status()

# Microsoft Fabric Details from Environment Variables
workspace_id = os.getenv("WORKSPACE_ID")
pipeline_id = os.getenv("PIPELINE_ID")

# Get the token using the class method
onelake_ops = OneLakeRemoteOperations()
token = onelake_ops.get_bearer_token()

# Trigger the pipeline job
job_location = trigger_pipeline_job(workspace_id=workspace_id,
                                    pipeline_id=pipeline_id,
                                    token=token)

print(f"Pipeline job triggered. Job location: {job_location}")

## **Run Delta Table Maintenance (REST API)**
**Reference:** [Background Jobs - Run On Demand Table Maintenance](https://learn.microsoft.com/en-us/rest/api/fabric/lakehouse/background-jobs/run-on-demand-table-maintenance?tabs=HTTP)

In [ ]:
%run ../main.py

def trigger_table_maintenance_job(table_name: str, token: str) -> str:
    endpoint = f"https://api.fabric.microsoft.com/v1/workspaces/{WORKSPACE_ID}/lakehouses/{LAKEHOUSE_ID}/jobs/instances?jobType=TableMaintenance"
    payload = {"executionData": {"tableName": table_name, "optimizeSettings": {"vOrder": True}, "vacuumSettings": {"retentionPeriod": "7:01:00:00"}}}
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    response = requests.post(endpoint, json=payload, headers=headers)
    if response.status_code == 202:
        return response.headers.get("Location")
    else:
        print(f"Failed to trigger table maintenance job. Status code: {response.status_code}, Response text: {response.text}")
        return None
    
def get_bearer_token() -> str:
    return InteractiveBrowserCredential().get_token("https://api.fabric.microsoft.com/.default").token

def get_authentication_token() -> DefaultAzureCredential:
    return DefaultAzureCredential()

#### **Single Table**

In [ ]:
%run ../main.py

# Get token
token = get_bearer_token() # Interactive Browser
# token = get_authentication_token().get_token("https://api.fabric.microsoft.com/.default").token # Service Principal

# Execute table maintenance job
trigger_table_maintenance_job(table_name="dim_coa_gold", token=token)

#### **All Tables**

In [ ]:
%run ../main.py
import time

# Get token
token = get_bearer_token() # Interactive Browser
# token = get_authentication_token().get_token("https://api.fabric.microsoft.com/.default").token # Service Principal
file_system_client = get_file_system_client(get_authentication_token())

# Get the filtered subdirectory names for "Tables"
filtered_tables = list_items(file_system_client=file_system_client, target_directory_path="Tables")

# Define batch size and delay between batches
batch_size = 5
batch_delay = 60  # in seconds

# Iterate over the filtered tables in batches
for i in range(0, len(filtered_tables), batch_size):
    batch_tables = filtered_tables[i:i + batch_size]
    for table_name in batch_tables:
        try:
            result = trigger_table_maintenance_job(table_name=table_name, token=token)
            if result is not None:
                print(f"Table maintenance job triggered for table: {table_name}")
            else:
                print(f"Failed to trigger table maintenance job for table: {table_name}")
        except Exception as e:
            print(f"An error occurred for table {table_name}: {e}")
    
    # Delay between batches
    if i + batch_size < len(filtered_tables):
        print(f"Waiting for {batch_delay} seconds before triggering the next batch...")
        time.sleep(batch_delay)